In [5]:
import json
import os
from ultralytics import YOLO



In [6]:
# 1. JSON bazangizni o'zgaruvchiga yuklaymiz
# (Yoki fayldan o'qish: data = json.load(open('food_info.json')))
food_db = [
    {"name": "CinnamonRolls", "total_base_kcal": 473, "ingredients": [{"item": "Xamir (un, sut)", "kcal": 250, "default": True}, {"item": "Sariyog'", "kcal": 100, "default": True}, {"item": "Shakar va dolchin", "kcal": 80, "default": True}, {"item": "Glazur (kremi)", "kcal": 43, "default": True}]},
    {"name": "Somsa", "total_base_kcal": 250, "ingredients": [{"item": "Xamir", "kcal": 120, "default": True}, {"item": "Go'sht (mol)", "kcal": 80, "default": True}, {"item": "Piyoz", "kcal": 10, "default": True}, {"item": "Dumba yog'i", "kcal": 40, "default": True}]},
    {"name": "Shorva", "total_base_kcal": 120, "ingredients": [{"item": "Go'shtli bulyon", "kcal": 50, "default": True}, {"item": "Qo'y go'shti", "kcal": 40, "default": True}, {"item": "Kartoshka", "kcal": 20, "default": True}, {"item": "Sabzi", "kcal": 10, "default": True}]},
    {"name": "KartoshkaChips", "total_base_kcal": 536, "ingredients": [{"item": "Kartoshka", "kcal": 300, "default": True}, {"item": "O'simlik yog'i", "kcal": 200, "default": True}, {"item": "Tuz va ziravorlar", "kcal": 36, "default": True}]},
    {"name": "Palov", "total_base_kcal": 330, "ingredients": [{"item": "Guruch", "kcal": 150, "default": True}, {"item": "Go'sht", "kcal": 100, "default": True}, {"item": "Sabzi", "kcal": 40, "default": True}, {"item": "Yog'", "kcal": 40, "default": True}, {"item": "Mayiz/No'xat", "kcal": 20, "default": False}]},
    {"name": "GrekSalati", "total_base_kcal": 115, "ingredients": [{"item": "Feta pishlog'i", "kcal": 50, "default": True}, {"item": "Zaytun yog'i", "kcal": 30, "default": True}, {"item": "Sabzavotlar", "kcal": 25, "default": True}, {"item": "Zaytun mevalari", "kcal": 10, "default": True}]},
    {"name": "Dolma", "total_base_kcal": 140, "ingredients": [{"item": "Uzum bargi", "kcal": 10, "default": True}, {"item": "Go'shtli qiyma", "kcal": 100, "default": True}, {"item": "Guruch", "kcal": 30, "default": True}, {"item": "Qatiq (sous)", "kcal": 10, "default": False}]},
    {"name": "Tovuq", "total_base_kcal": 239, "ingredients": [{"item": "Tovuq go'shti", "kcal": 180, "default": True}, {"item": "Yog' (qovurish)", "kcal": 40, "default": True}, {"item": "Ziravorlar", "kcal": 19, "default": True}]},
    {"name": "Pizza", "total_base_kcal": 250, "ingredients": [{"item": "Pitsa xamiri", "kcal": 120, "default": True}, {"item": "Pishloq (Motsarella)", "kcal": 80, "default": True}, {"item": "Sous va kolbasa", "kcal": 50, "default": True}, {"item": "Zaytun/Qo'ziqorin", "kcal": 16, "default": False}]},
    {"name": "Shashlik", "total_base_kcal": 220, "ingredients": [{"item": "Go'sht (mol/qo'y)", "kcal": 160, "default": True}, {"item": "Dumba yog'i", "kcal": 50, "default": True}, {"item": "Piyoz (garnir)", "kcal": 10, "default": True}]}
]



In [7]:
# 2. Modelni yuklash (best_int8.tflite fayli shu papkada bo'lishi kerak)
try:
    model = YOLO('best_int8.tflite', task='detect')
    print("✅ Model muvaffaqiyatli yuklandi.")
except Exception as e:
    print(f"❌ Modelni yuklashda xato: {e}")

def process_detection(image_path):
    # Model bashorati
    results = model.predict(source=image_path, conf=0.3)

    print(f"\n📸 Rasm tahlili: {image_path}")
    print("="*40)

    for result in results:
        if len(result.boxes) == 0:
            print("❓ Hech qanday taom aniqlanmadi.")
            return

        for box in result.boxes:
            # Model aniqlagan nomni olish
            detected_name = result.names[int(box.cls[0])]
            confidence = float(box.conf[0])

            print(f"🎯 Aniqlangan: {detected_name} ({confidence*100:.1f}%)")



✅ Model muvaffaqiyatli yuklandi.


In [11]:
# Redefine the complete process_detection function here to include the food database lookup logic
def process_detection(image_path):
    # Model bashorati
    results = model.predict(source=image_path, conf=0.3)

    print(f"\n📸 Rasm tahlili: {image_path}")
    print("="*40)

    for result in results:
        if len(result.boxes) == 0:
            print("❓ Hech qanday taom aniqlanmadi.")
            return

        for box in result.boxes:
            # Model aniqlagan nomni olish
            detected_name = result.names[int(box.cls[0])]
            confidence = float(box.conf[0])

            print(f"🎯 Aniqlangan: {detected_name} ({confidence*100:.1f}%)")

            # 3. JSON bazadan ma'lumotni qidirish
            # Nomlar mos kelishini tekshirish uchun kichik tuzatish
            food_info = next((f for f in food_db if f["name"].lower() == detected_name.lower()), None)

            if food_info:
                print(f"🔥 Umumiy kaloriya: {food_info['total_base_kcal']} kcal")
                print("\n🥗 Masalliqlar tarkibi:")

                for ing in food_info['ingredients']:
                    mark = "✅" if ing['default'] else "🔘"
                    print(f"   {mark} {ing['item']}: {ing['kcal']} kcal")
            else:
                print(f"⚠️ Ma'lumot topilmadi: {detected_name} bazada mavjud emas.")

            print("-" * 40)

# --- ISHLATISH ---
# Colab-ga rasm yuklang va nomini bu yerga yozing
process_detection('ovqat.jpg')#palov
process_detection('test1.jpg')#sinnabon
process_detection('test3.jpg')#do'lma


image 1/1 /content/ovqat.jpg: 640x640 1 Palov, 176.8ms
Speed: 4.3ms preprocess, 176.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

📸 Rasm tahlili: ovqat.jpg
🎯 Aniqlangan: Palov (99.0%)
🔥 Umumiy kaloriya: 330 kcal

🥗 Masalliqlar tarkibi:
   ✅ Guruch: 150 kcal
   ✅ Go'sht: 100 kcal
   ✅ Sabzi: 40 kcal
   ✅ Yog': 40 kcal
   🔘 Mayiz/No'xat: 20 kcal
----------------------------------------

image 1/1 /content/test1.jpg: 640x640 1 CinnamonRolls, 191.4ms
Speed: 3.3ms preprocess, 191.4ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

📸 Rasm tahlili: test1.jpg
🎯 Aniqlangan: CinnamonRolls (97.8%)
🔥 Umumiy kaloriya: 473 kcal

🥗 Masalliqlar tarkibi:
   ✅ Xamir (un, sut): 250 kcal
   ✅ Sariyog': 100 kcal
   ✅ Shakar va dolchin: 80 kcal
   ✅ Glazur (kremi): 43 kcal
----------------------------------------

image 1/1 /content/test3.jpg: 640x640 1 Dolma, 175.8ms
Speed: 3.8ms preprocess, 175.8ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)
